!pip install mlxtend

Import dependencies

In [ ]:
import pandas as pd

# Load your CSV data
data = pd.read_csv('transactions_list.csv')

# Assuming your CSV has a column named 'Transaction' where Transaction are separated by commas
data['Transaction'] = data['Transaction'].apply(lambda x: x.split(','))

# Create a list of transactions
transactions = data['Transaction'].tolist()

In [ ]:
class TreeNode:
    def __init__(self, name, count, parent=None):
        self.name = name
        self.count = count
        self.parent = parent
        self.children = {}

    def increment(self, count):
        self.count += count

class FPTree:
    def __init__(self, min_support):
        self.root = TreeNode('Root', 1)
        self.min_support = min_support
        self.header_table = {}

    def add_transaction(self, transaction):
        current_node = self.root
        for item in transaction:
            if item in current_node.children:
                current_node.children[item].increment(1)
            else:
                new_node = TreeNode(item, 1, current_node)
                current_node.children[item] = new_node
                if item in self.header_table:
                    self.header_table[item].append(new_node)
                else:
                    self.header_table[item] = [new_node]
            current_node = current_node.children[item]

    def create_tree(self, transactions):
        for transaction in transactions:
            self.add_transaction(transaction)

Frequent Transactions:
      support                         itemsets
0    0.082766                   (citrus fruit)
1    0.058566                      (margarine)
2    0.017692            (semi-finished bread)
3    0.139502                         (yogurt)
4    0.104931                 (tropical fruit)
..        ...                              ...
328  0.010168  (rolls/buns, frozen vegetables)
329  0.012405      (frozen vegetables, yogurt)
330  0.014235       (onions, other vegetables)
331  0.012100             (onions, whole milk)
332  0.010778      (sliced cheese, whole milk)

[333 rows x 2 columns]

Association Rules:
            antecedents      consequents  antecedent support  \
0        (citrus fruit)     (whole milk)            0.082766   
1          (whole milk)   (citrus fruit)            0.255516   
2        (citrus fruit)         (yogurt)            0.082766   
3              (yogurt)   (citrus fruit)            0.139502   
4      (tropical fruit)   (citrus fruit)         

In [ ]:
def mine_fp_tree(tree, prefix, frequent_itemsets):
    sorted_items = sorted(tree.header_table.items(), key=lambda x: x[1][0].count, reverse=True)
    for item, nodes in sorted_items:
        new_prefix = prefix + [item]
        support = sum(node.count for node in nodes)
        if support >= tree.min_support:
            frequent_itemsets.append((new_prefix, support))
            conditional_base = []
            for node in nodes:
                path = []
                parent = node.parent
                while parent and parent.name != 'Root':
                    path.append(parent.name)
                    parent = parent.parent
                conditional_base.extend([path] * node.count)
            conditional_tree = FPTree(tree.min_support)
            conditional_tree.create_tree(conditional_base)
            mine_fp_tree(conditional_tree, new_prefix, frequent_itemsets)

# Create the FP-tree
fp_tree = FPTree(min_support=2)  # Set your minimum support threshold
for transaction in transactions:
    fp_tree.create_tree([transaction])

# Mine the FP-tree
frequent_itemsets = []
mine_fp_tree(fp_tree, [], frequent_itemsets)

# Display the frequent itemsets
print("Frequent Itemsets:")
for itemset, support in frequent_itemsets:
    print(f"Itemset: {itemset}, Support: {support}")